In [ ]:
import sklearn
import numpy as np
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

True

In [ ]:
pip install cython

In [ ]:
pip install sent2vec

In [ ]:
pip install scikit-learn==0.22.2

In [ ]:
from sklearn.cluster import KMeans,AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import dendrogram
from sklearn.datasets import load_iris
from sklearn.mixture import GaussianMixture
from sklearn.cluster import SpectralClustering,DBSCAN
import sent2vec.vectorizer as vec

In [ ]:
import string

In [ ]:
from sklearn.datasets import fetch_20newsgroups

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Doc2Vec



In [ ]:
training_examples=fetch_20newsgroups(subset="train",remove=('headers','footers','quotes'))

In [ ]:
embeddings_index={}
glove_vocab=[]
with open('/content/drive/MyDrive/LEGAL_CLUSTERING/glove.6B.100d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        glove_vocab.append(word)
        embeddings_index[word] = coefs



In [ ]:
#Removing punctuation and tokenise and removing stopwords
def preprocessing(file):
  punctuationfree="".join([i for i in file if i not in string.punctuation])
  tokens=nltk.word_tokenize(punctuationfree)
  #stopwords = nltk.corpus.stopwords.words('english')
  #final_words= [i for i in tokens if i not in stopwords]
  return tokens

In [ ]:
data=[]
for x in training_examples.data:
  data.append(preprocessing(x))
documents=[]
y=[]
for a, d in enumerate(data):
  s=[]
  if len(d) <5:
    continue
  for i in d:
    try:
      l=embeddings_index[i.lower()]
      s.append(i.lower())
    except:
      continue
  y.append(training_examples.target[a])
  documents.append(TaggedDocument(s, [a]))

In [ ]:
def average_embed(data):
  embeddings=[]
  y=[]
  s=0
  for fil in range(len(data)):
    temp=np.zeros((100))
    for i in data[fil]:
      try:
        temp=temp+embeddings_index[i.lower()]
        s+=1
      except:
        continue
    if(np.sum(temp)!=0):
      temp=temp/s
      temp=temp/np.sqrt(np.sum(temp*temp))
      embeddings.append(temp.tolist())
      y.append(training_examples.target[fil])
  return (embeddings,y,s)

In [ ]:
def find_results(y,Y):
  answer={}
  l={}
  for i in range(len(training_examples.target_names)):
    answer[i]=[]
    l[i]=0
  for i in range(len(Y)):
    answer[Y[i]].append(i)
  sanswer=[]
  for i in answer:
    x=l.copy()
    for j in answer[i]:
      x[y[j]]=x[y[j]]+1
    #print(x)
    sanswer.append(x)
  return sanswer

In [ ]:
def acc(y_true, y_pred):
    """
    Calculate clustering accuracy. Require scikit-learn installed
    # Arguments
        y: true labels, numpy.array with shape `(n_samples,)`
        y_pred: predicted labels, numpy.array with shape `(n_samples,)`
    # Return
        accuracy, in [0,1]
    """
    y_true = y_true.astype(np.int64)
    assert y_pred.size == y_true.size
    D = max(y_pred.max(), y_true.max()) + 1
    w = np.zeros((D, D), dtype=np.int64)
    for i in range(y_pred.size):
        w[y_pred[i], y_true[i]] += 1
    from sklearn.utils.linear_assignment_ import linear_assignment
    ind = linear_assignment(w.max() - w)
    return sum([w[i, j] for i, j in ind]) * 1.0 / y_pred.size

# K-Means

In [ ]:
def initial_seeds(embeddings,y):
  samples=[]
  target=np.zeros(len(training_examples.target_names))
  for i in range(len(y)):
    if target[y[i]]==0:
      samples.append(embeddings[i])
      target[y[i]]=1
    if len(samples)==len(target):
      break;
  return samples

In [ ]:
def kmeans(embeddings,y):
  model = KMeans(n_clusters=len(training_examples.target_names),n_init=20)
  y_kmeans = model.fit_predict(embeddings)
  print(acc(np.array(y),y_kmeans))
  #print(find_results(y_kmeans))

In [ ]:
#Glove Average
embeddings,y,s=average_embed(data)

In [ ]:
kmeans(embeddings,y)

0.25136363636363634


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


# USING DOC2VEC FOR SENTENCE EMBEDDING

In [ ]:
"""
documents=[]
y=[]
for a, d in enumerate(data):
  s=[]
  if len(d) <5:
    continue
  for i in d:
    try:
      l=embeddings_index[i.lower()]
      s.append(i.lower())
    except:
      continue
  y.append(training_examples.target[a])
  documents.append(TaggedDocument(s, [a]))
"""
print(len(documents))

model1 = Doc2Vec(documents,dm=1, vector_size=300, window=8, min_count=1, workers=4,dbow_words=1)
model2=  Doc2Vec(documents,dm=1, vector_size=300, window=8, min_count=1, workers=4,dbow_words=0)
#Above model 1 and model2 are PVDM models(because dm=1) so using them either with dbow_words= '1' or '0' are same(i.e. no further skipgram training is done in these models)

model3=  Doc2Vec(documents,dm=0, vector_size=300, window=8, min_count=1, workers=4,dbow_words=1)
#In above model dbow_words=1 means skip gram fashion training is done along with PVDBOW training.

model4=  Doc2Vec(documents,dm=0, vector_size=300, window=8, min_count=1, workers=4,dbow_words=0)
#In above model dbow_words=0 means skip gram fashion training is not done along with PVDBOW training.


10891


In [ ]:
def doctwovec(model):
  embeddings=[model.infer_vector(i.words) for i in documents]
  for i in range(len(embeddings)):
    embeddings[i]/=np.sqrt(np.sum(embeddings[i]*embeddings[i]))
  kmeans(embeddings,np.array(y))


In [ ]:
doctwovec(model1)
doctwovec(model2)
doctwovec(model3)
doctwovec(model4)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


0.26581581122027365


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


0.26003121843724175


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


0.3717748599761271
0.27692590212101736


/usr/local/lib/python3.7/dist-packages/sklearn/utils/linear_assignment_.py:128: FutureWarning: The linear_assignment function is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


Above we can see that **model1** and **model3** give almost same accuracies.(As Expected).

**model3** gives highest accuracy which means that further training of predicted sentences in PVDBOW model in skip gram fashion provide best representation of words.

